In [ ]:
!rm -rf ../data/df_raw/*
!gsutil -m cp -r gs://np-training-tmp/stackoverflow/posts_processed/*.parquet "../data/df_raw

In [2]:
import modin.pandas as pd
import re
import lxml.html


In [ ]:
df_raw = pd.read_parquet("../data/df_raw")


In [ ]:
len(df_raw)

In [ ]:
def strip_html(s:str):
    try:
        return str(lxml.html.fromstring(s).text_content())
    except:
        return ''

def parse_tags(content:str):
    return re.findall(r'<(.+?)>',content)


In [ ]:
strip_html(f"""

<p>I was asked to create a singleton that will...	
""")

In [ ]:
df = df_raw

In [ ]:
df['Body'] =  df['Body'].apply(strip_html)

In [ ]:
df['Tags'] = df['Tags'].apply(parse_tags)

In [ ]:
df_final.to_parquet("../data/df_processed/")